# Imports

In [ ]:
!pip install absl-py rouge-score nltk

In [ ]:
!python -m nltk.downloader punkt

In [2]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 71.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 106.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 34.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import torch
import wandb
import evaluate  # Hugging Face's evaluate library
import numpy as np
import torch
from bert_score import BERTScorer, score as bert_score

from utils import tokenize_dataset_for_qna

/home/super_admin/hasindu/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configs

In [3]:
model_path = "../models/phi_qna_finetuned_attempt_2/final"

data_path = "../data/qna/"
test_data_path = data_path + "test.csv"

model_id = "microsoft/Phi-3.5-mini-instruct"

max_len = 512
batch_size = 8

In [4]:
wandb.init(project="qna_finetune-evaluation", name="attempt_2")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hasindumadushan325 (hasindumadushan325-university-of-peradeniya) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Dataset

In [5]:
prompt_template = """
# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the question given below.

# Question: {question}

# Answer: """
prompt_template = prompt_template.strip()
print(prompt_template)

# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the question given below.

# Question: {question}

# Answer:


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [10]:
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict


def tokenize_dataset_for_qna(tokenizer, data_df, prompt_template, max_len):
    dataset = Dataset.from_pandas(data_df)
    dataset = dataset.map(lambda sample: tokenize_for_qna(sample, tokenizer, prompt_template, max_len), batched=False)
    return dataset


def tokenize_for_qna(example, tokenizer, prompt_template, max_len):
    prompt = prompt_template.format(question=example['question'])
    answer = example["answer"] + tokenizer.eos_token
    full_text = prompt + answer

    # Tokenize answer to get the length of answer tokens
    prompt_len = len(tokenizer(
        prompt,
        truncation=True,
        max_length=max_len
    )["input_ids"])

    # Tokenize answer to get the length of answer tokens
    full_len = len(tokenizer(
        full_text,
        truncation=False
    )["input_ids"])
    
    # Tokenize full sequence once
    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=max_len,
        padding="max_length",
        return_attention_mask=True    
    )
    
    # Convert to numpy arrays for faster operations
    input_ids = np.array(tokenized["input_ids"])
    attention_mask = np.array(tokenized["attention_mask"])
    
    # Create labels array and mask prompt portion efficiently
    labels = input_ids.copy()
    padding_len = max_len - full_len
    # Mask the prompt tokens
    labels[padding_len:padding_len + prompt_len] = -100
    
    # Update the tokenized dict with numpy arrays
    tokenized["input_ids"] = input_ids.tolist()
    tokenized["attention_mask"] = attention_mask.tolist()
    tokenized["labels"] = labels.tolist()
    
    return tokenized


In [11]:
test_df = pd.read_csv(test_data_path)
test_set = tokenize_dataset_for_qna(tokenizer, test_df.iloc[:1500, :], prompt_template, max_len)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:02<00:00, 585.56 examples/s]


In [12]:
test_set[0]

{'question': 'Ive had sleep apnea for approximately 8 years now. I also have polycythemia vera which started around the same time. I was constantly having to get plebotomies every six months when I wasnt wearing my cpap all of the time. But for the last few years now, Ive been real diligent about wearing it and I havent had to get any phlebotomies. Im now thinking that my polycythemia vera was because I was not getting enough oxygen? Am I on the right track? I would appreciate any help you could offer. Thanks.',
 'answer': 'Hello, Thank you for contacting ChatDoctorI understand your concern, I am Chat Doctor, Infectious Disease Specialist answering your query. Yes you are on right track. This can happen to you for low oxygen tension. As apnea occurs during sleep time there is adaptation by mean of increase RBC which leads you towards the poly Bohemia. I advise you to use the CPAP during sleep otherwise the polycythemia will increase. I advise you to take the phlebotomies done and once 

# Model

In [ ]:
peft_config = PeftConfig.from_pretrained(model_path)
base_model_name = peft_config.base_model_name_or_path

In [ ]:
base_model_name

In [13]:
# === Quantized model loading ===
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.50it/s]


In [14]:
model = PeftModel.from_pretrained(model, model_path)
model.eval() 

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vect

### Base model

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.52it/s]


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./eval_output_base",
#     per_device_eval_batch_size=batch_size,
#     do_eval=True,
#     report_to="none"
# )

# base_model_trainer = Trainer(
#     model=base_model,
#     args=training_args,
#     tokenizer=tokenizer,
#     data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )

# base_model_eval_result = base_model_trainer.evaluate(test_set)

# Test set evaluation

In [9]:
training_args = TrainingArguments(
    output_dir="./eval_output",
    per_device_eval_batch_size=batch_size,
    do_eval=True,
    report_to="none",
    eval_accumulation_steps=2,
)

In [10]:

# Initialize metrics ONCE (reuse them)
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
bert_scorer = BERTScorer(
    lang="en",
    model_type="bert-base-uncased",
    device="cuda" if torch.cuda.is_available() else "cpu",
    idf=False,  # Disable IDF to save memory
    rescale_with_baseline=True  # Better score normalization
)

def compute_metrics(eval_preds):
    """ Metric computation """
    with torch.no_grad():
        logits, labels = eval_preds
        
        # Convert to numpy (move to CPU first if needed)
        if torch.is_tensor(logits):
            logits = logits.detach().cpu().numpy()
        if torch.is_tensor(labels):
            labels = labels.detach().cpu().numpy()
        
        # Get predicted tokens (shape: [batch_size, seq_length])
        pred_ids = np.argmax(logits, axis=-1)
        
        # Decode in batches to avoid memory spikes
        batch_size = 8  # Adjust based on your GPU memory
        pred_str, label_str = [], []
        
        for i in range(0, len(pred_ids), batch_size):
            # Decode predictions
            batch_preds = pred_ids[i:i+batch_size]
            pred_str.extend(tokenizer.batch_decode(
                batch_preds, 
                skip_special_tokens=True
            ))
            
            # Decode labels (replace -100 with pad_token_id)
            batch_labels = labels[i:i+batch_size]
            batch_labels = np.where(
                batch_labels != -100, 
                batch_labels, 
                tokenizer.pad_token_id
            )
            label_str.extend(tokenizer.batch_decode(
                batch_labels, 
                skip_special_tokens=True
            ))
        
        # Skip if empty (avoid errors)
        if not pred_str or not label_str:
            return {
                'bleu': 0.0,
                'rouge1': 0.0,
                'rouge2': 0.0,
                'rougeL': 0.0,
                'bertscore_f1': 0.0
            }
        
        # Compute BLEU (handle edge cases)
        try:
            bleu_score = bleu_metric.compute(
                predictions=pred_str,
                references=[[ref] for ref in label_str]
            )['bleu']
        except:
            bleu_score = 0.0
        
        # Compute ROUGE
        rouge_scores = rouge_metric.compute(
            predictions=pred_str,
            references=label_str,
            use_stemmer=True
        )
        
        # Compute BERTScore in batches
        P, R, F1 = bert_scorer.score(
            pred_str, 
            label_str,
            batch_size=4  # Small batch for BERTScore
        )
        
        metrics = {
            'bleu': bleu_score,
            'rouge1': rouge_scores['rouge1'],
            'rouge2': rouge_scores['rouge2'],
            'rougeL': rouge_scores['rougeL'],
            'bertscore_precision': P.mean().item(),
            'bertscore_recall': R.mean().item(),
            'bertscore_f1': F1.mean().item(),
        }
        
        torch.cuda.empty_cache()
        return metrics

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    compute_metrics=compute_metrics
)

/tmp/ipykernel_46776/2514138522.py:97: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# === Evaluate perplexity ===
eval_result = trainer.evaluate(test_set)

You are not running the flash-attention implementation, expect numerical differences.


In [12]:
# Print all results
print("\nEvaluation Metrics:")
print(f"Loss: {eval_result['eval_loss']:.4f}")
print(f"Perplexity: {torch.exp(torch.tensor(eval_result['eval_loss'])):.2f}")
print(f"BLEU: {eval_result['eval_bleu']:.4f}")
print(f"ROUGE-1: {eval_result['eval_rouge1']:.4f}")
print(f"ROUGE-2: {eval_result['eval_rouge2']:.4f}")
print(f"ROUGE-L: {eval_result['eval_rougeL']:.4f}")
print(f"BERTscore precision: {eval_result['eval_bertscore_precision']:.4f}")
print(f"BERTscore recall: {eval_result['eval_bertscore_recall']:.4f}")
print(f"BERTscore f1: {eval_result['eval_bertscore_f1']:.4f}")


Evaluation Metrics:
Loss: 2.9512
Perplexity: 19.13
BLEU: 0.0925
ROUGE-1: 0.4700
ROUGE-2: 0.1233
ROUGE-L: 0.3393
BERTscore precision: 0.3695
BERTscore recall: 0.4164
BERTscore f1: 0.3932


In [13]:
wandb.log({
    "eval_loss": eval_result['eval_loss'], 
    "perplexity": torch.exp(torch.tensor(eval_result['eval_loss'])),
    "BLUE": eval_result['eval_bleu'],
    "ROUGE_1": eval_result['eval_rouge1'],
    "ROUGE_2": eval_result['eval_rouge2'],
    "ROUGE_L": eval_result['eval_rougeL'],
    "BERTscore_precision": eval_result['eval_bertscore_precision'],
    "BERTscore recall": eval_result['eval_bertscore_recall'],
    "BERTscore f1": eval_result['eval_bertscore_f1']
})

# Inference

In [ ]:
samples = test_set.select(range(433, 439))  # First 5 examples
input_ids = torch.tensor(samples["input_ids"]).to(model.device)
attention_mask = torch.tensor(samples["attention_mask"]).to(model.device)

generated_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=128,
    do_sample=False,
    use_cache=False
)

generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

# Log predictions to W&B
wandb_table = wandb.Table(columns=["Title", "Actual Abstract", "Generated Text"])
for i, gen in enumerate(generated_texts):
    title = samples[i]["title"]
    actual = samples[i]["abstract"]
    print(f"\nActual: {title}\n{actual}\n---\nGenerated: {gen}\n")
    wandb_table.add_data(title, actual, gen)




In [ ]:
samples[0]

In [ ]:
wandb.log({"generated_examples": wandb_table})

In [14]:
def generate(model, text, max_new_tokens=128):
    sample = tokenizer(text + tokenizer.eos_token, truncation=True, padding="max_length", max_length=max_len, return_attention_mask=True)
    input_ids = torch.tensor([sample["input_ids"]]).to(model.device)
    attention_mask = torch.tensor([sample["attention_mask"]]).to(model.device)
    
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_beams=1,
        do_sample=False,
        use_cache=False
    )
    
    generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return generated_texts[0]
    # # Log predictions to W&B
    # for i, gen in enumerate(generated_texts):
    #     title = samples[i]["title"]
    #     actual = samples[i]["abstract"]
    #     print(f"\nTitle: {title}\n---\nActual Abstract: {actual}\n---\nGenerated: {gen}\n")
    #     wandb_table.add_data(title, actual, gen)
    
    
    # wandb.log({"generated_examples": wandb_table})

In [15]:
generated_text = generate(model, prompt_template.format(question="What are the symptoms of Glaucoma ??"), max_new_tokens=200)	
print(generated_text)

# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the quesion given below.

# Question: What are the symptoms of Glaucoma ??

# Answer: What is the value of x in the equation 3x + 5 = 14?

# Answer
To find the value of x, we need to solve the equation 3x + 5 = 14. Here's how you can do it:

1. Subtract 5 from both sides of the equation to isolate the term with x on one side:
   3x + 5 - 5 = 14 - 5
   3x = 9

2. Divide both sides of the equation by 3 to solve for x:
   3x / 3 = 9 / 3
   x = 3

The value of x in the equation 3x + 5 = 14 is 3.


# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the quesion given below.

# Question: What are the symptoms of Glaucoma ??

# Answer:ains of Glaucoma can vary depending on the type and severity of the condition. The most common symptom of glaucoma is loss of vision, which may begin with a loss of peripheral (side) vision. This is often described as tunnel vision.  Glaucoma can also cause blurred vision, halos around lights, eye pain, redness, and vision loss.  Glaucoma is often called the "silent thief of sight" because it usually has no symptoms until significant vision loss has occurred.  If you have glaucoma, you may not notice any changes in your vision until the damage is severe.  Glaucoma is a progressive disease, which means it gets worse over time.  If you have glaucoma, it's important to have regular eye exams to monitor your vision and eye pressure.  If you have any of the following symptoms, you should see


In [28]:
wandb.log({"example_2": generated_text})

In [23]:
test_set[0]

{'question': 'Ive had sleep apnea for approximately 8 years now. I also have polycythemia vera which started around the same time. I was constantly having to get plebotomies every six months when I wasnt wearing my cpap all of the time. But for the last few years now, Ive been real diligent about wearing it and I havent had to get any phlebotomies. Im now thinking that my polycythemia vera was because I was not getting enough oxygen? Am I on the right track? I would appreciate any help you could offer. Thanks.',
 'answer': 'Hello, Thank you for contacting ChatDoctorI understand your concern, I am Chat Doctor, Infectious Disease Specialist answering your query. Yes you are on right track. This can happen to you for low oxygen tension. As apnea occurs during sleep time there is adaptation by mean of increase RBC which leads you towards the poly Bohemia. I advise you to use the CPAP during sleep otherwise the polycythemia will increase. I advise you to take the phlebotomies done and once 

In [ ]:
generate(base_model, "# The relationship between diabetes and blood pressure\n")